In [2]:
import pandas as pd
import numpy as np
#for text pre-processing
import re, string
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import SnowballStemmer
from nltk.corpus import wordnet
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('wordnet')
nltk.download('stopwords')
nltk.download('punkt_tab')
nltk.download('averaged_perceptron_tagger_eng')
#for model-building
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import classification_report, f1_score, accuracy_score, confusion_matrix
from sklearn.metrics import roc_curve, auc, roc_auc_score
# bag of words
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import CountVectorizer
#for word embedding
import gensim
from gensim.models import Word2Vec

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.
[nltk_data] Downloading package averaged_perceptron_tagger_eng to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger_eng.zip.


In [5]:
df_train= pd.read_csv("/categories_test1.csv")
# Local path, may have to edit if using on a diff computer
df_test=pd.read_csv("/categories_test1.csv")

x=df_train['category'].value_counts()
print(x)

<ipython-input-5-412910a9f5a2>:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_train= pd.read_csv("/categories_test1.csv")


category
Pop Culture    226
History        209
Geography      158
Vocabulary     145
Literature     108
Science        103
Sports          43
Math             7
Name: count, dtype: int64


<ipython-input-5-412910a9f5a2>:3: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  df_test=pd.read_csv("/categories_test1.csv")


In [6]:
#convert to lowercase, strip and remove punctuations
def preprocess(text):
    text = text.lower()
    text=text.strip()
    text=re.compile('<.*?>').sub('', text)
    text = re.compile('[%s]' % re.escape(string.punctuation)).sub(' ', text)
    text = re.sub(r'\s+', ' ', text)
    text = re.sub(r'\[[0-9]*\]',' ',text)
    text=re.sub(r'[^\w\s]', '', str(text).lower().strip())
    text = re.sub(r'\d',' ',text)
    text = re.sub(r'\s+',' ',text)
    return text


# STOPWORD REMOVAL
def stopword(string):
    a= [i for i in string.split() if i not in stopwords.words('english')]
    return ' '.join(a)
#LEMMATIZATION
# Initialize the lemmatizer
wl = WordNetLemmatizer()

# This is a helper function to map NTLK position tags
def get_wordnet_pos(tag):
    if tag.startswith('J'):
        return wordnet.ADJ
    elif tag.startswith('V'):
        return wordnet.VERB
    elif tag.startswith('N'):
        return wordnet.NOUN
    elif tag.startswith('R'):
        return wordnet.ADV
    else:
        return wordnet.NOUN
# Tokenize the sentence
def lemmatizer(string):
    word_pos_tags = nltk.pos_tag(word_tokenize(string)) # Get position tags
    a=[wl.lemmatize(tag[0], get_wordnet_pos(tag[1])) for idx, tag in enumerate(word_pos_tags)] # Map the position tag and lemmatize the word/token
    return " ".join(a)

def finalpreprocess(string):
    return lemmatizer(stopword(preprocess(string)))

df_train['clean_text'] = df_train['question'].apply(lambda x: finalpreprocess(x))
print(df_train.head())

                                            question  question_id      answer  \
0  For the last 8 years of his life, Galileo was ...            0  Copernicus   
1  No. 2: 1912 Olympian; football star at Carlisl...            1  Jim Thorpe   
2  The city of Yuma in this state has a record av...            2     Arizona   
3  In 1963, live on "The Art Linkletter Show", th...            3  McDonald's   
4  Signer of the Dec. of Indep., framer of the Co...            4  John Adams   

      category  jeopardy_or_general  \
0      History                    0   
1       Sports                    0   
2    Geography                    0   
3  Pop Culture                    0   
4      History                    0   

                                          clean_text  
0  last year life galileo house arrest espouse ma...  
1  olympian football star carlisle indian school ...  
2  city yuma state record average hour sunshine year  
3  live art linkletter show company serve billion...  
4  

In [26]:
#SPLITTING THE TRAINING DATASET INTO TRAIN AND TEST
X_train, X_test, y_train, y_test = train_test_split(df_train["clean_text"],df_train["category"],train_size=900,shuffle=False)
#Word2Vec
# Word2Vec runs on tokenized sentences
X_train_tok= [nltk.word_tokenize(i) for i in X_train]
X_test_tok= [nltk.word_tokenize(i) for i in X_test]

#Tf-Idf
tfidf_vectorizer = TfidfVectorizer(use_idf=True)
X_train_vectors_tfidf = tfidf_vectorizer.fit_transform(X_train)
X_test_vectors_tfidf = tfidf_vectorizer.transform(X_test)
#building Word2Vec model
class MeanEmbeddingVectorizer(object):
    def __init__(self, word2vec):
        self.word2vec = word2vec
        # if a text is empty we should return a vector of zeros
        # with the same dimensionality as all the other vectors
        self.dim = len(next(iter(word2vec.values())))
    def fit(self, X, y):
        return self
    def transform(self, X):
        return np.array([
            np.mean([self.word2vec[w] for w in words if w in self.word2vec]
                    or [np.zeros(self.dim)], axis=0)
            for words in X
        ])

df_train['clean_text_tok']=[nltk.word_tokenize(i) for i in df_train['clean_text']]
model = Word2Vec(df_train['clean_text_tok'],min_count=1)
w2v = dict(zip(model.wv.index_to_key, model.wv.vectors))
modelw = MeanEmbeddingVectorizer(w2v)
# converting text to numerical data using Word2Vec
X_train_vectors_w2v = modelw.transform(X_train_tok)
X_val_vectors_w2v = modelw.transform(X_test_tok)

#FITTING THE CLASSIFICATION MODEL using Logistic Regression (W2v)
lr_w2v=LogisticRegression(solver = 'liblinear', C=10, penalty = 'l2')
lr_w2v.fit(X_train_vectors_w2v, y_train)  #model
#Predict y value for test dataset
y_predict = lr_w2v.predict(X_val_vectors_w2v)
y_prob = lr_w2v.predict_proba(X_val_vectors_w2v)[:,1]
# print(classification_report(y_test,y_predict))
print(y_predict)

#converting words to numerical data using tf-idf
X_vector=tfidf_vectorizer.transform(X_test)
#use the best model to predict 'target' value for the new dataset
#df_test['predict_prob']= y_prob
df_test.loc[900:, 'category'] = y_predict
final=df_test[['question','category']].reset_index(drop=True)
print(final.head())

['Geography' 'Vocabulary' 'Geography' ... 'History' 'Pop Culture'
 'Vocabulary']
                                            question     category
0  For the last 8 years of his life, Galileo was ...      History
1  No. 2: 1912 Olympian; football star at Carlisl...       Sports
2  The city of Yuma in this state has a record av...    Geography
3  In 1963, live on "The Art Linkletter Show", th...  Pop Culture
4  Signer of the Dec. of Indep., framer of the Co...      History


In [27]:
df_test.loc[1000:]

,question,question_id,answer,category,jeopardy_or_general
1000,This devoted mom has been called the most famo...,1000,Vanessa Williams,Geography,0
1001,From 1784 to 1788 the eastern part of this sta...,1001,Tennessee,Geography,0
1002,In 41 A.D. Cassius Chaerea & company assassina...,1002,Caligula,History,0
1003,This 1951 classic stars the AFI's top picks fo...,1003,The African Queen (Humphrey Bogart & Katharine...,Pop Culture,0
1004,"Fittingly, the book of Numbers begins with God...",1004,Moses,Literature,0
...,...,...,...,...,...
355309,"For a point each, name the 5 countries surroun...",1000138379,"Russia, China, Kyrgyzstan, Uzbekistan, and Tur...",Geography,1
355310,"March 9, 1959 saw the introduction of what Mat...",1000138380,Barbie,Pop Culture,1
355311,"On January 16, 2001, President Bill Clinton aw...",1000138381,Theodore Roosevelt,History,1
355312,"July 27, 1940 saw the introduction of what bel...",1000138382,Bugs Bunny,Pop Culture,1


In [29]:
df_test.to_csv('questions_with_categories.csv')